In [43]:
import torch
import torch.nn as nn
from torch.nn import functional as F

In [1]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [2]:
print("length of dataset in characters: ", len(text))

length of dataset in characters:  1115394


In [24]:
print(text[:462])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.


In [25]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [29]:
stoi = {s: i for i, s in enumerate(chars)}
itos = {i: s for s, i in stoi.items()}
encode = lambda s: [stoi[ch] for ch in s]
decode = lambda i: ''.join([itos[l] for l in i])

print(encode("MyDum"))
print(decode(encode("MyDum")))

[25, 63, 16, 59, 51]
MyDum


In [31]:
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:462])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [32]:
n = int(0.9*len(data))
train_data, val_data = data[:n], data[n:]

In [33]:
block_size = 8
batch_size = 4
torch.manual_seed(1337)

In [38]:
def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data)-block_size, (batch_size,))
    x = torch.stack([data[i: i + block_size] for i in ix])
    y = torch.stack([data[i + 1: i + block_size + 1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print(xb.shape)
print(xb)
print(yb.shape)
print(yb)

torch.Size([4, 8])
tensor([[ 1, 39, 52, 42,  1, 45, 43, 50],
        [ 1, 58, 46, 39, 58,  1, 42, 53],
        [ 1, 61, 53, 59, 50, 42,  1, 21],
        [59, 57, 40, 39, 52, 42,  1, 40]])
torch.Size([4, 8])
tensor([[39, 52, 42,  1, 45, 43, 50, 42],
        [58, 46, 39, 58,  1, 42, 53,  1],
        [61, 53, 59, 50, 42,  1, 21,  1],
        [57, 40, 39, 52, 42,  1, 40, 47]])


In [41]:
for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, :t+1]
        target = yb[b, t]
        print(f'When context is {context.tolist()} the target is {target}')

When context is [1] the target is 39
When context is [1, 39] the target is 52
When context is [1, 39, 52] the target is 42
When context is [1, 39, 52, 42] the target is 1
When context is [1, 39, 52, 42, 1] the target is 45
When context is [1, 39, 52, 42, 1, 45] the target is 43
When context is [1, 39, 52, 42, 1, 45, 43] the target is 50
When context is [1, 39, 52, 42, 1, 45, 43, 50] the target is 42
When context is [1] the target is 58
When context is [1, 58] the target is 46
When context is [1, 58, 46] the target is 39
When context is [1, 58, 46, 39] the target is 58
When context is [1, 58, 46, 39, 58] the target is 1
When context is [1, 58, 46, 39, 58, 1] the target is 42
When context is [1, 58, 46, 39, 58, 1, 42] the target is 53
When context is [1, 58, 46, 39, 58, 1, 42, 53] the target is 1
When context is [1] the target is 61
When context is [1, 61] the target is 53
When context is [1, 61, 53] the target is 59
When context is [1, 61, 53, 59] the target is 50
When context is [1, 61

In [48]:
class Bigram(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.emb = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):
        logits = self.emb(idx)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(-1)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, idx, nums):
        for _ in range(nums):
            logits, loss = self(idx)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx

In [49]:
model = Bigram(vocab_size)
logits, loss = model(xb, yb)
print(decode(model.generate(idx = torch.zeros((1, 1), dtype=torch.long), nums=100)[0].tolist()))


hYF SErndOyg.WvX-z QZVecAyrh:opPOPfo3VsY!-h:iwxT;vHspwvUiQZaO$fElV?Bov!
LYe3axQ'ME&o&l$YeW:cu:cg;zSl


In [50]:
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)

In [51]:
batch_size = 32

In [52]:
for steps in range(10000):
    xb, yb = get_batch('train')

    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

2.508171796798706


In [54]:
print(decode(model.generate(idx = torch.zeros((1, 1), dtype=torch.long), nums=500)[0].tolist()))


Bud mouar t ma, s s meero t keve ero wonoud BESis h l,
Whaimed sounddes, myor l
LAndease Wht pe ar,
LO:
LO,
WeLfthen?
Hur
O:Y:
Y tit eisee e Whentan myo stheerive fand a ts t.
Thooryothan
ATI'd il,

TI:

CARDnonghisil aun,
tou, y.


tthe; re, FQUn:

Dfe,
BEO,'torr ssentonknthes; terimempist!
T:
FineZKIsbewo;ar ghaze O
CEvart,
ARDot Thand ingie, ofongren.
WAR. y benvengarof c!
CKI wad, om'erd ICEOFer thandunsepl?
TYie aramy y meand lind ar t ge l ME:

Whis ay nten:

To mly u-fad
HOLallins me I th
